In [40]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

# **Part 1 - Data Preprocessing**

In [8]:
data = pd.read_csv('Churn_Modelling.csv')

# Eliminamos las columnas no importantes para el modelo son el RowNumber, CustomerId y el Surname
data = data[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']]

# Generamos la matriz de las variables independientes y el vector con los valores dependientes objetivo
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values

### **Encoding Categorical Data**

Se transforman las variables categoricas en numericas para que sean legibles por el modelo

In [14]:
# Utilizamos LabelEncoder para transformar la columna Gender en numérica
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [16]:
# Utilizamos One Hot Encoding para transformar la variable pais en numérica
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

### **Split the data into training and test**

Dividimos la matriz de variables independientes X y la matriz objetivo Y en una matriz de entrenamiento y una de test

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

### **Feature Scaling**

Normalizamos las variables de Train y Test

In [22]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# **Part 2 - Build the Artificial Neural Network**

### **Initializing the ANN**

Inicializamos la Red Neuronal

In [24]:
ann = tf.keras.models.Sequential()

### **Input and First Layer**

Add the input layer and the first layer

In [25]:
# Las variables a añadir a la capa input Dense es units: numero de neuronas (al ser input layer son el numero de features del modelo), activation: que funcion de activacion vamos a utilizar en este caso relu
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### **Second Layer**

In [27]:
# El numero de neuronas es elegido de forma aleatoria y optimizado una vez ejecutado el programa
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### **Output Layer**

In [28]:
# En este caso es 1 al ser una variable binaria, si fuera un resultado categorico con 3 posibles resultados habria que hacer OneHotEncoding a esos resultados y en número de unidades especificar 3
# Al ser un resultado binario la funcion de activacion sera sigmoid  
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# **Part 3 - Training the ANN**

In [29]:
# Compilamos la Red Neuronal
# La mejor loss function para binary clasification es binary crossentropy para non binary clasification debe ser softmax
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### **Training the ANN**

In [30]:
ann.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 803us/step - loss: 0.8615 - accuracy: 0.3767
Epoch 2/100
250/250 [==============================] - 0s 802us/step - loss: 0.5299 - accuracy: 0.7794
Epoch 3/100
250/250 [==============================] - 0s 803us/step - loss: 0.4657 - accuracy: 0.7965
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4339 - accuracy: 0.8107
Epoch 5/100
250/250 [==============================] - 0s 803us/step - loss: 0.4190 - accuracy: 0.8193
Epoch 6/100
250/250 [==============================] - 0s 762us/step - loss: 0.3977 - accuracy: 0.8300
Epoch 7/100
250/250 [==============================] - 0s 883us/step - loss: 0.3899 - accuracy: 0.8367
Epoch 8/100
250/250 [==============================] - 0s 836us/step - loss: 0.3760 - accuracy: 0.8396
Epoch 9/100
250/250 [==============================] - 0s 717us/step - loss: 0.3661 - accuracy: 0.8489
Epoch 10/100
250/250 [==============================] - 0s 816us/step - los

# **Part 4 - Making Predictions and Evaluating the Model**

In [37]:
# Debemos de aplicar StandardScaler para asegurar los datos son iguales que los de entranamiento
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000,2, 1, 1, 50000]]))

array([[0.06326926]], dtype=float32)

### **Predicting the test results**

Comprobamos como de bueno es nuestro modelo prediciendo los valores de test y comprobando el error con los originales

In [38]:
Y_pred = ann.predict(X_test)
y_pred = (Y_pred > 0.5)

In [42]:
## Utilizamos Confusion Matrix para comprobar como de buena es nuestra predicción

cm = confusion_matrix(Y_test, y_pred)
print(cm)
accuracy_score(Y_test, y_pred)

[[1545   50]
 [ 292  113]]


0.829